<a href="https://colab.research.google.com/github/Josiah-tan/py_misc/blob/main/param2attr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Param2attr
- The objective of this function is to automate creation of class attributes from parameters passed into an \_\_init\_\_ method
- Say we have a class that looks like this:

In [ ]:
class Foo:
  def __init__(self, param1, param2, param3):
    # This sux
    self.attr1 = param1
    self.attr2 = param2
    self.attr3 = param3

- We can instead create a class that looks like this, using a property decorator to perform the assignments

In [ ]:
class Foo:
  @param2attr(include="all", exclude=None)
  def __init__(self, param1, param2, param3):
    pass